# Spaceship Titanic Competition

### Goals: do some work with the categorical columns, one hot encode

To Do List - check if model can absorb nulls, if not convert floats to int (can't do that until there are no nulls)

In [2]:
import pandas as pd

final_data = pd.read_csv('test.csv')
train_data = pd.read_csv('train.csv')

In [3]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8492 non-null   object 
 2   CryoSleep     8476 non-null   object 
 3   Cabin         8494 non-null   object 
 4   Destination   8511 non-null   object 
 5   Age           8514 non-null   float64
 6   VIP           8490 non-null   object 
 7   RoomService   8512 non-null   float64
 8   FoodCourt     8510 non-null   float64
 9   ShoppingMall  8485 non-null   float64
 10  Spa           8510 non-null   float64
 11  VRDeck        8505 non-null   float64
 12  Name          8493 non-null   object 
 13  Transported   8693 non-null   bool   
dtypes: bool(1), float64(6), object(7)
memory usage: 891.5+ KB


In [90]:
train_data.VIP

0       False
1       False
2        True
3       False
4       False
        ...  
8688     True
8689    False
8690    False
8691    False
8692    False
Name: VIP, Length: 8693, dtype: object

setting features

In [70]:
features = train_data.columns.drop('Transported').to_list()
features

['PassengerId',
 'HomePlanet',
 'CryoSleep',
 'Cabin',
 'Destination',
 'Age',
 'VIP',
 'RoomService',
 'FoodCourt',
 'ShoppingMall',
 'Spa',
 'VRDeck',
 'Name']

In [71]:
from sklearn.model_selection import train_test_split

X = train_data[features]
y = train_data.Transported

X_train, X_valid, y_train, y_valid = train_test_split(X, y)

list of all categorical columns

In [72]:
cat_cols = X_train.select_dtypes(include='object').columns.to_list()
cat_cols

['PassengerId',
 'HomePlanet',
 'CryoSleep',
 'Cabin',
 'Destination',
 'VIP',
 'Name']

(code from the kaggle tutorial - checking to see how many unique values are in each categorical column)

In [73]:
# Get number of unique entries in each column with categorical data
object_nunique = list(map(lambda col: X_train[col].nunique(), cat_cols))
d = dict(zip(cat_cols, object_nunique))

# Print number of unique entries by column, in ascending order
sorted(d.items(), key=lambda x: x[1])

[('CryoSleep', 2),
 ('VIP', 2),
 ('HomePlanet', 3),
 ('Destination', 3),
 ('Cabin', 5168),
 ('Name', 6359),
 ('PassengerId', 6519)]

In [74]:
usecat = ['CryoSleep', 'VIP', 'HomePlanet', 'Destination']

In [75]:
from sklearn.preprocessing import OneHotEncoder

In [76]:
# Apply one-hot encoder to each column with categorical data
ohe = OneHotEncoder(handle_unknown='ignore', sparse=False)
ohe_train = pd.DataFrame(ohe.fit_transform(X_train[usecat]))
ohe_valid = pd.DataFrame(ohe.transform(X_valid[usecat]))

ohe_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6519 entries, 0 to 6518
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       6519 non-null   float64
 1   1       6519 non-null   float64
 2   2       6519 non-null   float64
 3   3       6519 non-null   float64
 4   4       6519 non-null   float64
 5   5       6519 non-null   float64
 6   6       6519 non-null   float64
 7   7       6519 non-null   float64
 8   8       6519 non-null   float64
 9   9       6519 non-null   float64
 10  10      6519 non-null   float64
 11  11      6519 non-null   float64
 12  12      6519 non-null   float64
 13  13      6519 non-null   float64
dtypes: float64(14)
memory usage: 713.1 KB


In [77]:
# One-hot encoding removed index; put it back
ohe_train.index = X_train.index
ohe_valid.index = X_valid.index

-------------------------------------------------------------------------------------------------------------------------------

setting up for imputation

In [78]:
num = train_data._get_numeric_data()
num.drop(['Transported'], axis=1, inplace=True)
numdata = num.columns.to_list()

slightly modified code from the kaggle tutorial

In [79]:
from sklearn.ensemble import RandomForestClassifier

(I'm just splitting up a cell here so I can see what is taking so long)

https://stackoverflow.com/questions/67361786/how-to-measure-random-forest-classifier-accuracy

In [80]:
from sklearn.impute import SimpleImputer
from sklearn import metrics

In [81]:
i = SimpleImputer()
i_X_train = pd.DataFrame(i.fit_transform(X_train[numdata]))
i_X_valid = pd.DataFrame(i.transform(X_valid[numdata]))

i_X_train.columns = X_train[numdata].columns
i_X_valid.columns = X_valid[numdata].columns

i_X_train.index = X_train.index
i_X_valid.index = X_valid.index

i_X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6519 entries, 5921 to 7958
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Age           6519 non-null   float64
 1   RoomService   6519 non-null   float64
 2   FoodCourt     6519 non-null   float64
 3   ShoppingMall  6519 non-null   float64
 4   Spa           6519 non-null   float64
 5   VRDeck        6519 non-null   float64
dtypes: float64(6)
memory usage: 356.5 KB


-------------------------------------------------------------------------------------------------------------------------------

In [82]:
# Add one-hot encoded columns to numerical features
both_X_train = pd.concat([i_X_train, ohe_train], axis=1)
both_X_valid = pd.concat([i_X_valid, ohe_valid], axis=1)
both_X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6519 entries, 5921 to 7958
Data columns (total 20 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Age           6519 non-null   float64
 1   RoomService   6519 non-null   float64
 2   FoodCourt     6519 non-null   float64
 3   ShoppingMall  6519 non-null   float64
 4   Spa           6519 non-null   float64
 5   VRDeck        6519 non-null   float64
 6   0             6519 non-null   float64
 7   1             6519 non-null   float64
 8   2             6519 non-null   float64
 9   3             6519 non-null   float64
 10  4             6519 non-null   float64
 11  5             6519 non-null   float64
 12  6             6519 non-null   float64
 13  7             6519 non-null   float64
 14  8             6519 non-null   float64
 15  9             6519 non-null   float64
 16  10            6519 non-null   float64
 17  11            6519 non-null   float64
 18  12            6519 non-nu

In [83]:
both_X_train.columns = both_X_train.columns.astype(str)
both_X_valid.columns = both_X_valid.columns.astype(str)

In [84]:
model = RandomForestClassifier(random_state=1, n_estimators=100, criterion='entropy')
model.fit(both_X_train, y_train)
preds = model.predict(both_X_valid)
print("Accuracy:", metrics.accuracy_score(y_valid, preds))

Accuracy: 0.7824287028518859


-------------------------------------------------------------------------------------------------------------------------------

In [85]:
# doing what I did to my data to all my training data
ohe_X = pd.DataFrame(ohe.transform(X[usecat]))
ohe_X.index = X.index

i_X = pd.DataFrame(i.transform(X[numdata]))
i_X.columns = X[numdata].columns
i_X.index = X.index

both_X = pd.concat([i_X, ohe_X], axis=1)
both_X.columns = both_X.columns.astype(str)

# i_X.info()

model.fit(both_X, y)

RandomForestClassifier(criterion='entropy', random_state=1)

In [86]:
ff = final_data[features]
ohe_final = pd.DataFrame(ohe.transform(ff[usecat]))
ohe_final.index = ff.index

i_X_final = pd.DataFrame(i.fit_transform(ff[numdata]))
i_X_final.columns = ff[numdata].columns
i_X_final.index = ff.index

both_X_final = pd.concat([i_X_final, ohe_final], axis=1)
both_X_final.columns = both_X.columns.astype(str)

In [88]:
final_preds = model.predict(both_X_final)

In [89]:
output = pd.DataFrame({'PassengerId': final_data.PassengerId, 'Transported': final_preds})
output.to_csv('submission3.csv', index=False)